In [9]:
import seaborn as sns
import pandas as pd
import json
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import collections
import os
import ast
import more_itertools as mit
from Bio.Seq import Seq

In [10]:
def finalQuickCheck(df):
    
    df2 = df.copy()
    
    for row in df2.index:
            
            if str(df2.at[row,'TE_Designation']) =='LINE/L1':
                continue
                    
            elif str(df2.at[row,'TE_Designation']) =='SINE/Alu':
                
                aluList = ast.literal_eval(str(df2.at[row,'TE_Hits']))
                if len(aluList)==1:
                    continue
                else:
                    
                    aluHitList=[]
                    otherElementList = []
                    for hit in aluList:
                        splitHit = hit.split()
                        if 'Alu' in str(splitHit[9]):
                            aluHitList.append(splitHit[9])
                            
                        elif 'SIMPLE' in str(splitHit[9]).upper():
                            continue
                            
                        else:
                            otherElementList.append(splitHit[9])
                            
                    if len(set(aluHitList))>1 or len(otherElementList)>0:
                        if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                            df2.at[row,'FILTER_RESULTS']=['MULTI-ALU_Hits']
                        else:
                            newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                            newList.append('MULTI-ALU_Hits')
                            df2.at[row,'FILTER_RESULTS']=newList

                    else:
                        continue

    return(df2)

In [11]:
replaceDict={'AG05252_PPY':'PonPyg',
 'AG06213_PAB':'PonAbe',
 'AG18354_PTR':'PanTro',
 'HG002':'HG002',
 'Human':'hs1',
 'Jim_GGO':'GorGor',
 'PR00251_PPA':'PanPan'}

elementReplace={'SINE/Alu':'Alu' , 'Retroposon/SVA':'SVA', 'LINE/L1':'L1'}

In [12]:
##sexChromosomes = ['CP068255.2','NC_060947.1','CP086569.2','NC_060948.1',
#                 'CM055469.2','CM055470.2','NC_073248.2','NC_011120.1','CM054605.2','CM054581.2',
#                 'CM054457.2','NC_072421.2','CM054458.2','NC_072422.2',
#                  'CM055495.2','NC_073272.2','CM055496.2','NC_073273.2','CM054483.2','CM054507.2',
#                  'CM054702.2','NC_072008.2','CM054703.2','NC_072009.2',
#                  'CM054653.2','NC_072396.2','CM054654.2','NC_072397.2'
#                 ]

In [13]:
from tqdm import tqdm
filteredDict={}
directory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Part4_FilteredandFixed/'
for csvFile in os.listdir(directory):
    if '_Final' in str(csvFile):
        print(csvFile)
        species = str(csvFile.split("_")[0])
        filteredDict[species]={'Unique':{'SINE/Alu': 0,'LINE/L1': 0,'Retroposon/SVA': 0}, 'Fixed':{'SINE/Alu': 0,'LINE/L1': 0,'Retroposon/SVA': 0}}

        inDF1 = pd.read_csv(directory+csvFile).set_index("ID")
        df2 = finalQuickCheck(inDF1)
        df1 = df2[df2['FILTER_RESULTS']=='Good_Row'].copy()

        goodRows=[]
        for row in df1.index:
            if str(df1.at[row,'TE_Designation'])=='SINE/Alu' and float(df1.at[row,'TE_Percentage'])>=.70:
                goodRows.append(row)
            elif str(df1.at[row,'TE_Designation'])=='LINE/L1' and float(df1.at[row,'TE_Percentage'])>=.20:
                goodRows.append(row)

            elif str(df1.at[row,'TE_Designation'])=='Retroposon/SVA' and float(df1.at[row,'TE_Percentage'])>=.20:
                goodRows.append(row)

            else:
                continue

        df = df1.loc[goodRows].copy()
        uniqueelementDict = collections.Counter(df[df['Fixed']=='NO']['TE_Designation'])
        nonuniqueelementDict = collections.Counter(df[df['Fixed']!='NO']['TE_Designation'])

        for key in uniqueelementDict:
            if key in filteredDict[species]['Unique'].keys():
                filteredDict[species]['Unique'][key]=uniqueelementDict[key]
            else:
                continue
        for key in nonuniqueelementDict:
            if key in filteredDict[species]['Fixed'].keys():
                filteredDict[species]['Fixed'][key]=nonuniqueelementDict[key]
            else:
                continue
                
        for element in set(df['TE_Designation']):
            tempDF = df[df['TE_Designation']==element].copy()
            
            
            
            if element == 'LINE/L1':
                elementName = 'LINE'
            elif element == 'Retroposon/SVA':
                elementName ='SVA'
            else:
                elementName = 'Alu'
                    
            #with open('/home/mark/Desktop/TE_Leaves/ls_primates/Cactus_MEI_Species_Sequences/'+str(species)+"_"+str(elementName)+".fasta", 'a+') as file:
            #    for row in tempDF.index:
            #        elementName2 = str(tempDF.at[row,'Element_Annotation'])
            #        file.write('>'+str(row)+"_"+elementName2+'\n')
            #        if str(tempDF.at[row,'Tail_Type']) == 'Possible_T-Tail' or str(tempDF.at[row,'Tail_Type']) == 'Possible_A-Tail_and_Possible_T-Tail*':
            #            file.write(str(tempDF.at[row,'Sequence'])+"\n")
            #        else:
            #            sequence2 = str(Seq(tempDF.at[row,'Sequence']).reverse_complement())
            #            file.write(sequence2+"\n")
            #file.close()
        
        #df.to_csv('/home/mark/Desktop/TE_Leaves/ls_primates/Final_Dataframes/Cactus/'+str(species)+"_Final_07-05-2024.csv")
                        
            
    else:
        continue

PonPyg_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv
PanPan_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv
PonAbe_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv
hs1_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv
GorGor_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv
PanTro_FinalTEPercentageFiltered_FixedAnnotation_06-26-2024.csv


In [5]:
tempList=[]
for species in filteredDict.keys():
    for element in filteredDict[species]['Unique']:
        if element == 'snRNA':
            continue
        else:
            tempList.append([species, element, int(filteredDict[species]['Unique'][element]), 'Unique'])
            tempList.append([species, element, int(filteredDict[species]['Fixed'][element]), 'Fixed'])

In [14]:
tempList=[]
for species in filteredDict.keys():
    for element in filteredDict[species]['Unique']:
        if element == 'snRNA':
            continue
        else:
            tempList.append([species, elementReplace[element], int(filteredDict[species]['Unique'][element]), 'Unique'])
            tempList.append([species, elementReplace[element], int(filteredDict[species]['Fixed'][element]), 'Fixed'])

In [15]:
tempDF = pd.DataFrame(data=tempList, columns=['Species', 'Element', 'Count','Classification'])
tempDF

,Species,Element,Count,Classification
0,PonPyg,Alu,55,Unique
1,PonPyg,Alu,49,Fixed
2,PonPyg,L1,1941,Unique
3,PonPyg,L1,1325,Fixed
4,PonPyg,SVA,356,Unique
5,PonPyg,SVA,190,Fixed
6,PanPan,Alu,379,Unique
7,PanPan,Alu,1708,Fixed
8,PanPan,L1,347,Unique
9,PanPan,L1,804,Fixed


In [16]:
sorter=['PanPan','PanTro','hs1','GorGor','PonAbe','PonPyg','SymSyn']
#sorter2=['SINE/Alu' , 'Retroposon/SVA', 'LINE/L1']
sorter2=['Alu' , 'SVA', 'L1']

tempDF.sort_values(by="Species", key=lambda column: column.map(lambda e: sorter.index(e)), inplace=True)

In [17]:
tempDF2 = tempDF[tempDF['Species']!='SymSyn'].copy()
tempDF2['Reference']='Cactus'

In [11]:
#tempDF2.to_csv('/home/mark/Desktop/TE_Leaves/ls_primates/dataframes/CU_TEleaves_Counts.csv')

In [18]:
alt.Chart(tempDF2).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('Element:N', title=None, sort=sorter2, axis=alt.Axis(labelAngle=0)),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(Count):Q',
        axis=alt.Axis(
            grid=True,
            title='Total MEIs', titleFont='Arial')).scale(domain=(0, 10000)),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('Species:N',title='Cactus Unaligned Dataset - Species', sort=sorter),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Classification:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['#4793AF', '#FFC470'],
            ),
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0  
    
    
    ).properties(
    padding={"left": 20, "top": 20, "right": 20, "bottom": 20}
).properties(width=75, height=300).configure_header(
    # Configure header options
    labelFontSize=12  # Adjust the font size for column title
)


alt.Chart(...)

In [19]:
alt.Chart(tempDF2).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('Element:N', title=None, sort=sorter2, axis=alt.Axis(labelAngle=0)),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(Count):Q',
        axis=alt.Axis(
            grid=True,
            title='Total MEIs')).scale(domain=(0, 10000)),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('Species:N',title='Cactus Unaligned Dataset - Species', sort=sorter),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Element:N',legend=None,
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['green', '#FFC470', '#4793AF'],
            ), sort=sorter2,
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    ).properties(
    padding={"left": 20, "top": 20, "right": 20, "bottom": 20}
).properties(width=75, height=300).configure_header(
    # Configure header options
    labelFontSize=12  # Adjust the font size for column title
)

alt.Chart(...)

In [20]:
divegentMYR={'HG002':6.2,'hs1':6.2,'PanPan':1.6,'PanTro':1.6,'GorGor':10.7,'PonPyg':.96, 'PonAbe':.96}

In [21]:
tableCountDf = pd.read_csv('/home/mark/Desktop/TE_Leaves/FullLengthElement_Count_WSexChromosomes_07-08-2024.csv').drop(columns=['Unnamed: 0'])
tableCountDf['Normalized_Total_Elements'] = [int(int(x)/divegentMYR[y]) for x,y in zip(tableCountDf['Total_Elements'], tableCountDf['Species'])]

In [19]:
#tableCountDf.set_index("Species").to_csv('/home/mark/Desktop/LS_Element_Count_Dataframe-07-12-2024.csv')

In [15]:
#divegentMYR={'hs1':4.5,'PanPan':1,'PanTro':1,'GorGor':6,'PonPyg':1, 'PonAbe':1}

In [22]:
insperMY= tempDF2.copy()

In [23]:
insperMY['Count_Per_Mya']=0
for row in insperMY.index:
    species = str(insperMY.at[row,'Species'])
    insperMY.at[row,'Count_Per_Mya']=int(insperMY.at[row,'Count'])/divegentMYR[species]

In [27]:
insperMY

,Species,Element,Count,Classification,Reference,Count_Per_Mya
11,PanPan,SVA,433,Fixed,Cactus,270.625000
10,PanPan,SVA,286,Unique,Cactus,178.750000
9,PanPan,L1,804,Fixed,Cactus,502.500000
8,PanPan,L1,347,Unique,Cactus,216.875000
7,PanPan,Alu,1708,Fixed,Cactus,1067.500000
6,PanPan,Alu,379,Unique,Cactus,236.875000
34,PanTro,SVA,212,Unique,Cactus,132.500000
35,PanTro,SVA,434,Fixed,Cactus,271.250000
30,PanTro,Alu,345,Unique,Cactus,215.625000
31,PanTro,Alu,1586,Fixed,Cactus,991.250000


In [26]:
alt.Chart(insperMY).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('Element:N', title=None, sort=sorter2, axis=alt.Axis(labelAngle=0)),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(Count_Per_Mya):Q',
        axis=alt.Axis(
            grid=True,
            title='MEIs per Millions of Years (Mya)')).scale(domain=(0, 3500)),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('Species:N', title='Cactus Unaligned Dataset - Species', sort=sorter),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Element:N',legend=None,
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['green', '#FFC470', '#4793AF'],
            ), sort=sorter2,
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    ).properties(
    padding={"left": 20, "top": 20, "right": 20, "bottom": 20}
).properties(width=75, height=300).configure_header(
    # Configure header options
    labelFontSize=12  # Adjust the font size for column title
)

alt.Chart(...)

In [2]:
df = pd.read_csv('/home/mark/Desktop/TE_Leaves/LS_Element_Count_Dataframe-07-12-2024.csv')

In [5]:
df2 = df[df['DataSet'].str.contains("Cactus", regex=True)].copy()

In [6]:
df2

,Species,Fixed_FL_Elements,Total_Fixed_Elements,FL_Elements,Total_Elements,DataSet,Normalized_Total_Elements
0,PanTro,1245,1586,1499,1931,Cactus_Alu,1135
1,PanTro,156,922,214,1241,Cactus_L1,730
2,PanTro,283,434,408,646,Cactus_SVA,380
3,PanPan,1330,1708,1620,2087,Cactus_Alu,1227
4,PanPan,176,804,257,1151,Cactus_L1,677
5,PanPan,280,433,463,719,Cactus_SVA,422
6,PonAbe,24,33,60,86,Cactus_Alu,86
7,PonAbe,257,942,810,3201,Cactus_L1,3201
8,PonAbe,71,113,253,410,Cactus_SVA,410
9,GorGor,4038,4841,4969,5986,Cactus_Alu,486
